In [10]:
import pyomo.environ as pe
import pyomo.opt as po

### Problema Car Renting 

Se plantea como variables binarias. 
xij = 1 o 0 . En cada pais i, cada coche j, elijo o no el coche (variable binaria)

Parámetros: 
- coste de alquiler de cada coche en cada país 

Restricciones (todas las lógicas preposicionales del problema)
- coste de 25€ por cada cambio 
- un coche por país (solo puede valer 1 coche 1, y los otros 2 deben valer 0 )
- empieza en españa y termina en italia 



Función objetivo : minimizar los costes 


In [11]:
modelo = pe.ConcreteModel()

SETS 

In [12]:
modelo.paises = pe.Set(initialize=['Spain', 'France', 'Germany', 'Austria', 'Switzerland','Italy'])
modelo.coches = pe.Set(initialize=['Car1', 'Car2', 'Car3'])



PARAMETROS

In [13]:

combustible_dict = {
    ('Spain','Car1'):160, ('France','Car1'):210, ('Germany','Car1'):180, ('Austria','Car1'):110, ('Switzerland','Car1'):85,  ('Italy','Car1'):170,
    ('Spain','Car2'):120, ('France','Car2'):240, ('Germany','Car2'):165, ('Austria','Car2'):135, ('Switzerland','Car2'):100, ('Italy','Car2'):160,
    ('Spain','Car3'):150, ('France','Car3'):200, ('Germany','Car3'):175, ('Austria','Car3'):140, ('Switzerland','Car3'):115, ('Italy','Car3'):135,
}
modelo.combustible_coche_pais = pe.Param(modelo.paises, modelo.coches, initialize=combustible_dict)
modelo.coste_cambio = pe.Param(initialize=25)

VARIABLES

In [14]:
#variable que vale 1 si se ha elegido ese coche en ese pais y 0 en caso contrario
modelo.coche_pais = pe.Var(modelo.paises, modelo.coches, within=pe.Binary)
#variable binaria qeu vale 1 si se ha cambiado de coche entre el pais anterior y este y 0 en caso contrario
modelo.cambio_pais = pe.Var(['France', 'Germany', 'Austria', 'Switzerland','Italy'], within=pe.Binary)

FUNCIO OBJETIVO



In [15]:
def funcion_objetivo(modelo):
    combustible = sum(modelo.combustible_coche_pais[i, k] * modelo.coche_pais[i, k] for i in modelo.paises for k in modelo.coches)
    cambios = sum(modelo.coste_cambio * modelo.cambio_pais[i] for i in modelo.cambio_pais)
    return combustible + cambios

modelo.coste = pe.Objective(rule=funcion_objetivo, sense=pe.minimize)


RESTRICCIONES

In [16]:
#restriccion de un solo coche por pais
def un_coche_por_pais(modelo, i):
    return sum(modelo.coche_pais[i, k] for k in modelo.coches) == 1

modelo.AsignacionUnica = pe.Constraint(modelo.paises, rule=un_coche_por_pais)

#restriccion de cambio de coche
diccionario_paises_anterior = {'France':'Spain', 'Germany':'France', 'Austria':'Germany', 'Switzerland':'Austria', 'Italy':'Switzerland'}
# aplicamos la restriccion cambio_pais[pais] ≥ coche_pais[pais,k] − coche_pais[pais_anterior,k]
def cambio_mayor_igual(modelo, coche, pais):
    if pais not in modelo.cambio_pais:
        return pe.Constraint.Skip
    # detecta 0→1
    return modelo.cambio_pais[pais] >= modelo.coche_pais[pais, coche] - modelo.coche_pais[diccionario_paises_anterior[pais], coche]

modelo.cambio_mayor_igual = pe.Constraint(modelo.coches, modelo.paises, rule=cambio_mayor_igual)





RESOLVER


In [17]:
solucionador = po.SolverFactory('gurobi') 
resultado = solucionador.solve(modelo, tee=False)



RESULTADO


In [18]:
paises_orden = list(modelo.paises.data())
coches_orden = list(modelo.coches.data())

itinerario = []
for i in paises_orden:
    k_best = max(coches_orden, key=lambda k: pe.value(modelo.coche_pais[i, k]))
    itinerario.append((i, k_best))

coste_total = pe.value(modelo.coste)
num_cambios = sum(int(round(pe.value(modelo.cambio_pais[i]))) for i in modelo.cambio_pais.index_set())

print("plan optimo:")
for pais, coche in itinerario:
    print(f"{pais}:{coche}")
print(f"coste total: {int(round(coste_total))} €")
print(f"numero de cambios: {num_cambios} (penalización: {num_cambios * pe.value(modelo.coste_cambio)} €)")

plan optimo:
Spain:Car2
France:Car1
Germany:Car1
Austria:Car1
Switzerland:Car1
Italy:Car3
coste total: 890 €
numero de cambios: 2 (penalización: 50 €)
